In [9]:
import os
import cv2
import numpy as np

# Ruta al directorio de datos
data_dir = 'C:\\datasets\\archive\\train'

# Lista de etiquetas de emociones
emotions = ['happy', 'sad', 'surprise', 'angry', 'disgust', 'fear', 'neutral']

# Dimensiones deseadas de las imágenes
img_size = (48, 48)

# Inicializar listas para las imágenes y etiquetas
faces = []
labels = []

# Función para cargar imágenes en lotes
def load_images_batch(emotion_dir, emotion_label, batch_size=100):
    batch_faces = []
    batch_labels = []
    img_files = os.listdir(emotion_dir)[:200]  # Limitar a 100 imágenes por categoría
    
    for i, img_name in enumerate(img_files):
        img_path = os.path.join(emotion_dir, img_name)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        if img is not None:
            img = cv2.resize(img, img_size)  # Redimensionar la imagen
            batch_faces.append(img)
            batch_labels.append(emotion_label)
        else:
            print(f"Error al cargar la imagen {img_path}")
        
        # Procesar en lotes
        if (i + 1) % batch_size == 0 or (i + 1) == len(img_files):
            yield np.array(batch_faces), np.array(batch_labels)
            batch_faces, batch_labels = [], []

# Cargar las imágenes y etiquetas en lotes
for emotion in emotions:
    emotion_dir = os.path.join(data_dir, emotion)
    if not os.path.exists(emotion_dir):
        print(f"La carpeta {emotion_dir} no existe, revisa la ruta.")
        continue

    for batch_faces, batch_labels in load_images_batch(emotion_dir, emotions.index(emotion)):
        faces.extend(batch_faces)
        labels.extend(batch_labels)

# Convertir las listas a arrays de numpy
faces = [img.reshape(img_size[0], img_size[1], 1) for img in faces]
faces = np.array(faces)
labels = np.array(labels)

# Normalizar las imágenes
faces = faces.astype('float32') / 255.0

print(f"Cargadas {faces.shape[0]} imágenes con dimensiones {faces.shape[1:]}")

# Dividir en entrenamiento y prueba
from sklearn.model_selection import train_test_split

faces_train, faces_test, labels_train, labels_test = train_test_split(faces, labels, test_size=0.2, random_state=42)

print(f"Conjunto de entrenamiento: {faces_train.shape[0]} imágenes")
print(f"Conjunto de prueba: {faces_test.shape[0]} imágenes")


# Crear el modelo de Eigenfaces
model = cv2.face.EigenFaceRecognizer_create()

# Convertir las etiquetas a formato adecuado
labels_train = np.array(labels_train, dtype=np.int32)
labels_test = np.array(labels_test, dtype=np.int32)

# Aplanar las imágenes para el entrenamiento
faces_train_flatten = [img.flatten() for img in faces_train]
faces_test_flatten = [img.flatten() for img in faces_test]

# Entrenar el modelo
model.train(np.array(faces_train_flatten), labels_train)

# Guardar el modelo entrenado
model.save('modelo_eigenfaces7.xml')







Cargadas 1400 imágenes con dimensiones (48, 48, 1)
Conjunto de entrenamiento: 1120 imágenes
Conjunto de prueba: 280 imágenes


## Training 2.0

In [2]:

import os
import cv2 as cv
import numpy as np

data_dir = 'C:\\datasets\\archive\\train'
faces  = os.listdir(data_dir)
print(faces)

labels = []
facesData = []
label = 0 
for face in faces:
    facePath = data_dir+'\\'+face # Construye la ruta completa
    for faceName in os.listdir(facePath)[:300]: # Toma los primeros 300 archivos
        labels.append(label)
        facesData.append(cv.imread(facePath+'/'+faceName,0)) # Lee la imagen en escala de grises
    label = label + 1
print(np.count_nonzero(np.array(labels)==0)) 

faceRecognizer = cv.face.EigenFaceRecognizer_create()
faceRecognizer.train(facesData, np.array(labels))
faceRecognizer.save('modelo_eigenfaces8.xml')  

['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']
300


In [3]:
import os
import cv2 as cv
import numpy as np

# Cargar el modelo entrenado
faceRecognizer = cv.face.EigenFaceRecognizer_create()
faceRecognizer.read('modelo_eigenfaces9.xml')
#data_dir = 'C:\\datasets\\archive\\train'
#faces  = os.listdir(data_dir)
faces = ["feliz", "sorprendido", "triste"]
print(faces)

cap = cv.VideoCapture(0)
rostro = cv.CascadeClassifier(cv.data.haarcascades + 'haarcascade_frontalface_alt.xml')

while True:
    ret, frame = cap.read()
    if not ret:break

    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    cpGray = gray.copy()

    rostros = rostro.detectMultiScale(gray, 1.3, 3)

    for (x, y, w, h) in rostros:
        frame2 = cpGray[y:y + h, x:x + w]
        frame2 = cv.resize(frame2, (100, 100), interpolation=cv.INTER_CUBIC)
        result = faceRecognizer.predict(frame2)

        if result[1] > 3800: #3800
            cv.putText(frame, '{}'.format(faces[result[0]]), (x, y - 25), 2, 1.1, (0, 255, 0), 1, cv.LINE_AA)
            cv.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        else:
            cv.putText(frame, 'Neutral', (x, y - 20), 2, 0.8, (0, 0, 255), 1, cv.LINE_AA)
            cv.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)

    cv.imshow('frame', frame)
    # Salir del bucle si se presiona la tecla 'q'
    if cv.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv.destroyAllWindows()





['feliz', 'sorprendido', 'triste']


In [1]:
import cv2

# Verificar si el módulo face de cv2 está disponible
try:
    face_recognizer = cv2.face.EigenFaceRecognizer_create()
    print("opencv-contrib-python está correctamente instalado.")
except AttributeError:
    print("opencv-contrib-python NO está instalado o no se ha importado correctamente.")


opencv-contrib-python está correctamente instalado.


In [1]:
import cv2 as cv
img = cv.imread("./assets/fire/fire.13.png")

cv.imshow("Display window", img)
k = cv.waitKey(0) # Wait for a keystroke in the window

: 